<a href="https://colab.research.google.com/github/chiragmohod/AutoHire/blob/main/AutoHire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 1: Install required packages
!pip install -q sentence-transformers PyPDF2 scikit-learn pandas

# STEP 2: Upload JD CSV and resume PDFs
from google.colab import files
import os

print("📁 Upload your job_description.csv file:")
jd_file = files.upload()

print("\n📄 Upload your resume PDF files:")
resume_files = files.upload()

os.makedirs("resumes", exist_ok=True)
for filename in resume_files:
    with open(f"resumes/{filename}", "wb") as f:
        f.write(resume_files[filename])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.2 MB/s eta 0:00:00
📁 Upload your job_description.csv file:


Saving job_description.csv to job_description.csv

📄 Upload your resume PDF files:


Saving C1061.pdf to C1061.pdf
Saving C1070.pdf to C1070.pdf
Saving C1080.pdf to C1080.pdf
Saving C1161.pdf to C1161.pdf
Saving C1164.pdf to C1164.pdf
Saving C1191.pdf to C1191.pdf
Saving C1212.pdf to C1212.pdf
Saving C1228.pdf to C1228.pdf
Saving C1236.pdf to C1236.pdf
Saving C1320.pdf to C1320.pdf
Saving C1446.pdf to C1446.pdf
Saving C1487.pdf to C1487.pdf
Saving C1499.pdf to C1499.pdf
Saving C1547.pdf to C1547.pdf
Saving C1627.pdf to C1627.pdf
Saving C1677.pdf to C1677.pdf
Saving C1701.pdf to C1701.pdf
Saving C1781.pdf to C1781.pdf
Saving C1789.pdf to C1789.pdf
Saving C1791.pdf to C1791.pdf
Saving C1796.pdf to C1796.pdf
Saving C2098.pdf to C2098.pdf
Saving C2139.pdf to C2139.pdf
Saving C2144.pdf to C2144.pdf
Saving C2235.pdf to C2235.pdf
Saving C2250.pdf to C2250.pdf
Saving C2256.pdf to C2256.pdf
Saving C2287.pdf to C2287.pdf
Saving C2430.pdf to C2430.pdf
Saving C2546.pdf to C2546.pdf
Saving C2603.pdf to C2603.pdf
Saving C2607.pdf to C2607.pdf
Saving C2652.pdf to C2652.pdf
Saving C26

In [2]:
# STEP 3: Read and summarize JDs with Titles
import pandas as pd

jd_df = pd.read_csv(next(iter(jd_file)))
print("📄 Job Descriptions Loaded:\n", jd_df.head())

# Identify title & description columns
title_col = [col for col in jd_df.columns if 'title' in col.lower()][0]
desc_col = [col for col in jd_df.columns if 'description' in col.lower()][0]

# Combine both for better semantic embedding
combined_jds = (jd_df[title_col] + " - " + jd_df[desc_col]).fillna("").tolist()
job_titles = jd_df[title_col].fillna("Unknown Role").tolist()


📄 Job Descriptions Loaded:
                Job Title                                    Job Description
0      Software Engineer   Description:\nWe are seeking a skilled Softwa...
1         Data Scientist  Job Description:\nWe are looking for a skilled...
2        Product Manager  Description:\nWe are seeking an innovative and...
3         Cloud Engineer  Description:\nWe are looking for a skilled Clo...
4  Cybersecurity Analyst  Description:\nWe are looking for a skilled Cyb...


In [3]:
# STEP 4: Extract text from resumes
import PyPDF2

def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

resumes_text = []
resume_names = []

for file in os.listdir("resumes"):
    if file.endswith(".pdf"):
        resume_text = extract_text_from_pdf(f"resumes/{file}")
        resumes_text.append(resume_text)
        resume_names.append(file)


In [4]:
from sentence_transformers import SentenceTransformer, util

print("📊 Embedding JDs and Resumes...\n")
model = SentenceTransformer('all-MiniLM-L6-v2')

jd_embeddings = model.encode(combined_jds, convert_to_tensor=True)
resume_embeddings = model.encode(resumes_text, convert_to_tensor=True)

print("📊 Matching Resumes with JDs...\n")
threshold = 0.40  # You can adjust this

shortlist = []  # For console viewing
matches = {}    # For Mistral or automation

for i, combined_jd in enumerate(combined_jds):
    print(f"\n🔹 JD {i+1} ({job_titles[i]}):\n{combined_jd[:100]}...\n")
    scores = util.cos_sim(jd_embeddings[i], resume_embeddings)[0]

    jd_matches = []  # Store match info for current JD

    for j, score in enumerate(scores):
        candidate_data = {
            "resume": resume_names[j],
            "score": float(score),
            "resume_text": resumes_text[j]
        }

        if score >= threshold:
            print(f"✅ {resume_names[j]} - Match Score: {score:.2f}")
            shortlist.append({
                "resume": resume_names[j],
                "job_title": job_titles[i],
                "score": float(score)
            })
            jd_matches.append(candidate_data)
        else:
            print(f"❌ {resume_names[j]} - Match Score: {score:.2f}")

    matches[job_titles[i]] = jd_matches


📊 Embedding JDs and Resumes...



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

📊 Matching Resumes with JDs...


🔹 JD 1 (Software Engineer):
Software Engineer -  Description:
We are seeking a skilled Software Engineer to design, develop, and...

✅ C7554.pdf - Match Score: 0.54
✅ C7543.pdf - Match Score: 0.46
✅ C7733.pdf - Match Score: 0.42
✅ C5786.pdf - Match Score: 0.50
✅ C1164.pdf - Match Score: 0.55
✅ C9677.pdf - Match Score: 0.49
✅ C1677.pdf - Match Score: 0.53
✅ C3821.pdf - Match Score: 0.55
✅ C3363.pdf - Match Score: 0.56
✅ C4602.pdf - Match Score: 0.50
✅ C3761.pdf - Match Score: 0.45
❌ C6658.pdf - Match Score: 0.39
✅ C9369.pdf - Match Score: 0.52
✅ C9142.pdf - Match Score: 0.53
✅ C1796.pdf - Match Score: 0.53
✅ C3627.pdf - Match Score: 0.42
✅ C4336.pdf - Match Score: 0.41
✅ C8518.pdf - Match Score: 0.46
✅ C7775.pdf - Match Score: 0.47
✅ C5390.pdf - Match Score: 0.42
✅ C7482.pdf - Match Score: 0.41
✅ C4021.pdf - Match Score: 0.44
✅ C8203.pdf - Match Score: 0.52
✅ C8239.pdf - Match Score: 0.58
✅ C2775.pdf - Match Score: 0.48
✅ C7230.pdf - Match Score: 0.52
✅ 

In [5]:
from collections import defaultdict

print("📊 Displaying Top 5 Resumes for Each Job Description...\n")

# Group results by JD
matches = defaultdict(list)
for entry in shortlist:
    matches[entry["job_title"]].append({
        "resume": entry["resume"],
        "score": entry["score"]
    })

# Sort and display top 5
for jd_title, candidates in matches.items():
    print(f"\n🧠 Job Title: {jd_title}\n" + "-"*50)
    # Sort candidates by score
    top_matches = sorted(candidates, key=lambda x: x["score"], reverse=True)[:5]

    for idx, candidate in enumerate(top_matches, start=1):
        print(f"{idx}. {candidate['resume']} - Score: {candidate['score']:.2f}")


📊 Displaying Top 5 Resumes for Each Job Description...


🧠 Job Title: Software Engineer
--------------------------------------------------
1. C8886.pdf - Score: 0.63
2. C7784.pdf - Score: 0.63
3. C3315.pdf - Score: 0.62
4. C6746.pdf - Score: 0.61
5. C2607.pdf - Score: 0.60

🧠 Job Title: Data Scientist
--------------------------------------------------
1. C9342.pdf - Score: 0.67
2. C9897.pdf - Score: 0.66
3. C5756.pdf - Score: 0.65
4. C9369.pdf - Score: 0.65
5. C8631.pdf - Score: 0.65

🧠 Job Title: Product Manager
--------------------------------------------------
1. C3539.pdf - Score: 0.49
2. C1499.pdf - Score: 0.48
3. C9742.pdf - Score: 0.48
4. C4460.pdf - Score: 0.48
5. C7492.pdf - Score: 0.47

🧠 Job Title: Cloud Engineer
--------------------------------------------------
1. C8730.pdf - Score: 0.70
2. C8692.pdf - Score: 0.69
3. C6592.pdf - Score: 0.68
4. C9142.pdf - Score: 0.68
5. C1796.pdf - Score: 0.68

🧠 Job Title: Cybersecurity Analyst
--------------------------------------------

In [6]:
import sqlite3

# Connect to SQLite DB (creates file if it doesn't exist)
conn = sqlite3.connect("matches.db")
cursor = conn.cursor()

# Create table to store matches
cursor.execute("""
CREATE TABLE IF NOT EXISTS resume_matches (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    job_title TEXT,
    resume_name TEXT,
    score REAL
)
""")
conn.commit()


In [7]:
# Insert shortlisted matches into DB
for entry in shortlist:
    cursor.execute("""
    INSERT INTO resume_matches (job_title, resume_name, score)
    VALUES (?, ?, ?)
    """, (entry["job_title"], entry["resume"], entry["score"]))

conn.commit()


In [8]:
job_titles_in_db = cursor.execute("SELECT DISTINCT job_title FROM resume_matches").fetchall()

for (job_title,) in job_titles_in_db:
    print(f"\n🧠 Job Title: {job_title}\n" + "-"*50)

    top_resumes = cursor.execute("""
    SELECT resume_name, score
    FROM resume_matches
    WHERE job_title = ?
    ORDER BY score DESC
    LIMIT 5
    """, (job_title,)).fetchall()

    for idx, (resume, score) in enumerate(top_resumes, start=1):
        print(f"{idx}. {resume} - Score: {score:.2f}")



🧠 Job Title: Software Engineer
--------------------------------------------------
1. C8886.pdf - Score: 0.63
2. C7784.pdf - Score: 0.63
3. C3315.pdf - Score: 0.62
4. C6746.pdf - Score: 0.61
5. C2607.pdf - Score: 0.60

🧠 Job Title: Data Scientist
--------------------------------------------------
1. C9342.pdf - Score: 0.67
2. C9897.pdf - Score: 0.66
3. C5756.pdf - Score: 0.65
4. C9369.pdf - Score: 0.65
5. C8631.pdf - Score: 0.65

🧠 Job Title: Product Manager
--------------------------------------------------
1. C3539.pdf - Score: 0.49
2. C1499.pdf - Score: 0.48
3. C9742.pdf - Score: 0.48
4. C4460.pdf - Score: 0.48
5. C7492.pdf - Score: 0.47

🧠 Job Title: Cloud Engineer
--------------------------------------------------
1. C8730.pdf - Score: 0.70
2. C8692.pdf - Score: 0.69
3. C6592.pdf - Score: 0.68
4. C9142.pdf - Score: 0.68
5. C1796.pdf - Score: 0.68

🧠 Job Title: Cybersecurity Analyst
--------------------------------------------------
1. C4507.pdf - Score: 0.66
2. C4999.pdf - Score: 

In [9]:
import sqlite3

conn = sqlite3.connect("matches.db")
cursor = conn.cursor()

job_title = input("Enter Job Title: ")

# Fetch top 5 resumes for that job title
cursor.execute("""
SELECT resume_name, score
FROM resume_matches
WHERE job_title = ?
ORDER BY score DESC
LIMIT 5
""", (job_title,))
results = cursor.fetchall()

print(f"\nTop 5 resumes for '{job_title}':")
for i, (resume, score) in enumerate(results, start=1):
    print(f"{i}. {resume} - Score: {score:.2f}")

conn.close()


Enter Job Title: Robotics Engineer

Top 5 resumes for 'Robotics Engineer':
1. C9945.pdf - Score: 0.54
2. C9590.pdf - Score: 0.54
3. C6746.pdf - Score: 0.54
4. C7784.pdf - Score: 0.54
5. C3922.pdf - Score: 0.54


In [10]:
from collections import defaultdict

top_matches_by_jd = defaultdict(list)

for match in shortlist:
    top_matches_by_jd[match['job_title']].append(match)

# Sort and trim to top 5
for job_title in top_matches_by_jd:
    top_matches_by_jd[job_title] = sorted(top_matches_by_jd[job_title], key=lambda x: x['score'], reverse=True)[:5]


In [11]:
def generate_interview_email(candidate_name, job_title, interview_date, interview_time, mode):
    return f"""\
Subject: Interview Invitation for {job_title} Role

Dear {candidate_name},

We are pleased to inform you that you have been shortlisted for the {job_title} position based on your resume.

We would like to invite you for an interview scheduled on {interview_date} at {interview_time}, to be conducted in {mode} mode.

Kindly confirm your availability by replying to this email. We look forward to speaking with you.

Best regards,
Recruitment Team
AutoHire AI
"""

# Example: generating for top 5 of a JD
from datetime import datetime, timedelta

interview_date = (datetime.today() + timedelta(days=2)).strftime("%B %d, %Y")
interview_time = "11:00 AM"
mode = "online"

for jd_title, matches in top_matches_by_jd.items():
    print(f"\n📩 Emails for Job Title: {jd_title}\n" + "-"*50)
    for candidate in matches:
        name = candidate["resume"].replace(".pdf", "").replace("_", " ").title()
        email = generate_interview_email(name, jd_title, interview_date, interview_time, mode)
        print(email)
        print("-" * 50)



📩 Emails for Job Title: Software Engineer
--------------------------------------------------
Subject: Interview Invitation for Software Engineer Role

Dear C8886,

We are pleased to inform you that you have been shortlisted for the Software Engineer position based on your resume.

We would like to invite you for an interview scheduled on April 09, 2025 at 11:00 AM, to be conducted in online mode.

Kindly confirm your availability by replying to this email. We look forward to speaking with you.

Best regards,
Recruitment Team
AutoHire AI

--------------------------------------------------
Subject: Interview Invitation for Software Engineer Role

Dear C7784,

We are pleased to inform you that you have been shortlisted for the Software Engineer position based on your resume.

We would like to invite you for an interview scheduled on April 09, 2025 at 11:00 AM, to be conducted in online mode.

Kindly confirm your availability by replying to this email. We look forward to speaking with you